In [1]:
# Importing required Libraries

import pandas as pd
import numpy as np

In [2]:
recipe = pd.read_csv('../Measurements&nutrition_edited_final_tables/recipe_measurements_edited.csv')
NUT_CPLT = pd.read_csv('../Measurements&nutrition_edited_final_tables/nut_edited.csv')

In [6]:
recipe.head()

,recipe_no,ingredient_Phrase,ingredient,state,quantity,unit,temp,df,size,ing_id,ndb_id,M_or_A
0,2610,3 cups water,water,NaN,3,cup,NaN,NaN,NaN,3,14555,M
1,2610,1 cup red lentils,red lentil,NaN,1,cup,NaN,NaN,NaN,452,16144,A
2,2610,"1 roma tomato , quartered",rom tomato,quartered,1,unknown,NaN,NaN,NaN,180,93600,A
3,2610,"1 carrot , quartered",carrot,quartered,1,unknown,NaN,NaN,NaN,21,11124,M
4,2610,"1 small onion , quartered",onion,quartered,1,small,NaN,NaN,small,1,11282,M


In [3]:
NUT_CPLT['description'] = NUT_CPLT['description'].astype(str) + " ("+ NUT_CPLT['units']+")"


In [4]:
NUT_CPLT.drop(['tag_name','nutrient_no','units'], axis =1,inplace = True)
recipe.drop(['df','temp','size','ingredient_Phrase','ingredient','state','ing_id','M_or_A'], axis =1,inplace = True)

In [5]:
complt = pd.merge(NUT_CPLT, recipe,  how='inner', left_on=['ndb_id','unit'], right_on = ['ndb_id','unit'])
complt.head()

,ndb_id,nutrient_val,description,unit,amount,grams,recipe_no,quantity
0,1003,0.28,Protein (g),tbsp,1.0,12.8,2741,1
1,1003,0.28,Protein (g),tbsp,1.0,12.8,2934,2
2,1003,0.28,Protein (g),tbsp,1.0,12.8,3037,NaN
3,1003,0.28,Protein (g),tbsp,1.0,12.8,3281,NaN
4,1003,0.28,Protein (g),tbsp,1.0,12.8,3285,NaN


In [6]:
del recipe
del NUT_CPLT

In [7]:
# function which will convert a string like 2/5 or 1 1/5 to a float 
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac
def conv(frac_str):
    try:
        return convert_to_float(frac_str)
    except ValueError:
        return 0
    
#applying above function to the Quantity column    
complt['quantity'] = complt['quantity'].apply(conv)

#converting data to float so it can be computed
complt[['grams','amount','quantity','nutrient_val']] = complt[['grams','amount','quantity','nutrient_val']].astype(float)
complt['nutrition_final'] = (complt.grams)*(complt.quantity)/(complt.amount)*(complt.nutrient_val)/100
complt['nutrition_final'] = complt["nutrition_final"].round(4)
# complt['gms'] = (complt.grams)*(complt.quantity)/(complt.amount)

# converting final value of nutrition to string and concatonating with units
# complt['nutrition_final'] = complt['nutrition_final'].astype(str) + " "+ complt['units'] 

In [8]:
# reordering some columns, and this is the final dataset  
complt = complt[['recipe_no','ndb_id','description','nutrition_final','unit']]
complt = complt.rename(columns={'nutrition_final': 'nutrition'})


In [9]:
complt.to_csv('./Final_unpivoted.csv',index=None)

In [ ]:
complt =pd.read_csv('Final_unpivoted.csv')

In [ ]:
complt.head()

In [ ]:
complt[complt['quantity']>200]